# Background

Ideas:

1. Give users who have older accounts more weight in calculating the optimal timestamps
2. Guve users who used more words in the twitch chat more weight

## User Stats

In [41]:
def user_stats(dataframe):
    '''
    Returns the user who sent the most messages, most emoticons, and who changed their username the most
    '''
    temp_df = dataframe.groupby("name").count().reset_index()
    temp_df['username_chg'] = temp_df['_id'] - temp_df['display_name'] # if mismatch in count, then its probs changed username
    
    stats = {}
    for col in ['_id','emoticons', 'username_chg']:
        sort = temp_df.sort_values(col, ascending=False)[['name',col]].iloc[0]
        stat = sort[col]
        _id = sort['name']
        
        if col == '_id':
            col = 'num_messages'
        stats[col] = [_id, stat]
    
    return stats

In [42]:
user_stats(big_df)

{'num_messages': ['gassyvegan247', 95],
 'emoticons': ['gassyvegan247', 14],
 'username_chg': ['148u', 0]}